In [66]:
import xml.etree.ElementTree as ET
import duckdb
import os
from duckdb import ConstraintException
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import re

In [67]:
# Connect to sql database
con = duckdb.connect(database='german-parliament', read_only=False)

In [68]:
def parse_xml(file_path):
    """
    Parses an XML file and returns the root element.

    Args:
        file_path (str): The path to the XML file.

    Returns:
        xml.etree.ElementTree.Element: The root element of the parsed XML tree,
                                         or None if parsing fails.
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        return root
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return None
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

In [69]:
path = "../data/plenary_minutes/wahlperiode_19/19_001_2017-10-24.xml"
root = parse_xml(path)
# Get metadata
kopfdaten = root.find("vorspann").find("kopfdaten")
# Get period and session no
plenarprotokoll_nummer = kopfdaten.find("plenarprotokoll-nummer")
period = plenarprotokoll_nummer.find("wahlperiode").text
session = plenarprotokoll_nummer.find("sitzungsnr").text
# Get the date
raw_date = kopfdaten.find("veranstaltungsdaten").find('datum').attrib.get("date")
date = datetime.strptime(raw_date, "%d.%m.%Y")
# print(attributes)
print(ET.tostring(verantstaltungsdaten,encoding="unicode"))
print(f"Period: {period}, Session: {session}, Date: {date}")

<veranstaltungsdaten><ort>Berlin</ort>, <datum date="24.10.2017">Dienstag, den 24. Oktober 2017</datum></veranstaltungsdaten>
    
Period: 19, Session: 1, Date: 2017-10-24 00:00:00


In [83]:
tagesordnungspunkte = root.find("sitzungsverlauf").findall("tagesordnungspunkt")
for top in tagesordnungspunkte:
    top_id_text = top.attrib.get("top-id")
    # Extract the numeric part from the top-id attribute
    match = re.search(r"\d+$", top_id_text)
    top_id = int(match.group()) if match else None
    print(f"Top ID: {top_id}")
    # extract the speeches for the give ordnungspunkt
    reden = top.findall('rede')
    for rede in reden:
        speech_id = rede.attrib.get('id')
        speaker = rede.find('p').find('redner')
        speaker_id = speaker.attrib.get('id')
        speaker_data = speaker.find('name')
        # Extract the speaker's first and last name
        speaker_first_name = speaker_data.find('vorname').text
        speaker_last_name = speaker_data.find('nachname').text
        # Extract the speaker role if it exists. e.g. "Alterspräsident"
        speaker_position = speaker_data.find('rolle')
        if speaker_position is not None:
            speaker_position = speaker_rolle.find("rolle_lang").text
        else:
            speaker_position = None
        # Extract the speakers party
        speaker_party = speaker_data.find('fraktion')
        if speaker_party is not None:
            speaker_party = speaker_party.text
        else:
            speaker_party = None

        print(f"Speech ID: {speech_id}, Speaker ID: {speaker_id}, "
              f"Name: {speaker_first_name} {speaker_last_name}, "
              f"Role: {speaker_rolle}, Party: {speaker_party}")

        speach_text = ET.tostring(rede,encoding="unicode")


Top ID: 1
Speech ID: ID19100100, Speaker ID: 11002190, Name: Alterspräsident Dr. Hermann Otto Solms, Role: Alterspräsident, Party: None
<rede id="ID19100100">
        <p klasse="redner">

        <redner id="11002190"><name><vorname>Alterspräsident Dr. Hermann</vorname><nachname>Otto Solms</nachname><rolle><rolle_lang>Alterspräsident</rolle_lang><rolle_kurz>Alterspräsident</rolle_kurz></rolle></name></redner>

        Alterspräsident Dr. Hermann Otto Solms:</p>
        <p klasse="J_1">Guten Morgen, liebe Kolleginnen und Kollegen! Nehmen Sie bitte Platz.</p>
        <p klasse="J">Meine sehr verehrten Damen und Herren! Liebe Kolleginnen und Kollegen! Ich begrüße Sie zur konstituierenden Sitzung des 19. Deutschen Bundestages. Es entspricht der ständigen Übung, zu Beginn der konstituierenden Sitzung nach den Regelungen der bisherigen Geschäftsordnung des Deutschen Bundestages zu verfahren.</p>
        <p klasse="J">§ 1 Absatz 2 der Geschäftsordnung des Deutschen Bundestages sieht vor, dass